# Query SIMBAD and find J and K band magnitudes 

In [1]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

from astroquery.simbad import Simbad
from astroquery.vizier import Vizier

from astropy import coordinates
from astropy.coordinates import EarthLocation, SkyCoord
from astropy.time import Time
import astropy.units as u

from astroplan import Observer
from astroplan import FixedTarget
from astroplan.plots import plot_airmass

from pytz import timezone


>>> from astroplan import download_IERS_A
>>> download_IERS_A()
 [astroplan.utils]


# Import database using pandas

In [6]:

db = pd.read_csv('../../../datasets/Accretion Rate Database - Brown Dwarfs.csv', delimiter=',')
db = db.dropna(how='all')
db.head()

,Source,Star Forming Region,COORDS,Disk Type,Reference,Epoch,Effective Temperature K,Temp err,Sp Type,Method,...,Age range Myr,System Age Myr,Companion?,Notes/Questions,Notes/Questions2,K mag,K mag err,J mag,J mag err,Simbad name
0,2M J11013205-7718249,Chamaeleon I,165.3833 -77.3069,II,Muzerolle 2005,30-Dec-03,2710.0,NaN,M8,Echelle Spectra,...,NaN,2.0,NaN,NaN,NaN,13.540,0.040,14.631,0.039,ESO-HA 554
1,2MASS 1207-3932,TW HyA,181.75 -39.533,U,Mohanty 2005,May 2002 - December 2003,2700.0,NaN,M8.0,High Res spectra,...,NaN,8.0,NaN,upper limit,Accretion rate equation 1,11.945,0.026,12.995,0.026,TWA 27
2,2MASS 1207-3932,TW HyA,181.75 -39.533,U,Mohanty 2005,May 2002 - December 2003,2700.0,NaN,M8.0,High Res spectra,...,NaN,8.0,NaN,upper limit,Accretion rate equation 3,11.945,0.026,12.995,0.026,TWA 27
3,2MASS J04141188+2811535,Tarus,63.549505 28.198196,II,Muzerolle 2005,30-Dec-03,2962.0,NaN,M6.25,Echelle Spectra,...,NaN,1.0,NaN,NaN,disk classification from LUHMAN ET AL. JAN 2010,11.639,0.021,13.161,0.024,2MASS J04141188+2811535
4,2MASS J04381486+2611399,Tarus,69.56192 26.194427,II,Muzerolle 2005,30-Dec-03,2838.0,NaN,M7.25,Echelle Spectra,...,NaN,1.0,NaN,NaN,disk classification from LUHMAN ET AL. JAN 2010,12.980,0.030,15.180,0.050,2MASS J04381486+2611399


# Query K and J band magnitudes

### Note: Some inputs might give multiple results - the querying will find more than 1 object.   Check results against RA/Dec and Main ID.   

In [144]:
def query_simbad(obj, coordinates, radius = 10):
    '''
    inputs:
        obj: str -- name of object to query
        coordinates: astropy.coordinates object
        radius: unit -- distance in arcseconds to search around coordinates for object 
    outputs:
        None: if querying does not find anything
        results_table[['FLUX_K', 'FLUX_ERROR_K','FLUX_J', 'FLUX_ERROR_J' ]]: array of K and J band magnitude and flux
    '''

    print('\nobject: ' + obj + '\nRA, DEC: ' + str(coordinates.to_string('hmsdms')) + '\n')
        
    # initialize SIMBAD and find values of interest
    customSimbad = Simbad()
    customSimbad.add_votable_fields('flux(K)')
    customSimbad.add_votable_fields('flux_error(K)')
    customSimbad.add_votable_fields('flux(J)')
    customSimbad.add_votable_fields('flux_error(J)')

    # search though SIMBAD within radius arcseconds around given coordinate
    result_table = customSimbad.query_region(c, radius=radius*u.arcsecond)
    if result_table is None:
        print('No results found with SIMBAD')
        print('_______________________________________ o ________________________________________')
        return None
    else:
        #print(result_table)
        print('SIMBAD results:')
        print(result_table[['MAIN_ID','RA', 'DEC', 'FLUX_K', 'FLUX_ERROR_K','FLUX_J', 'FLUX_ERROR_J']])
        print('_______________________________________ o ________________________________________')
        return result_table[['FLUX_K', 'FLUX_ERROR_K','FLUX_J', 'FLUX_ERROR_J' ]]
    


    

# Individually


In [145]:
obj = '2M J11013205-7718249'
obj_row = db.loc[db['Source'] == obj]

c = coordinates.SkyCoord(obj_row['COORDS'].values, frame='icrs', unit='deg')

RESULTS = query_simbad(obj, c)



object: 2M J11013205-7718249
RA, DEC: ['11h01m31.992s -77d18m24.84s']

SIMBAD results:
  MAIN_ID         RA           DEC      FLUX_K FLUX_ERROR_K FLUX_J FLUX_ERROR_J
               "h:m:s"       "d:m:s"     mag                 mag               
----------- ------------- ------------- ------ ------------ ------ ------------
ESO-HA  554 11 01 32.0412 -77 18 25.075  13.54        0.040 14.631        0.039
_______________________________________ o ________________________________________


## To access values in RESULTS

In [146]:
print('to get table column: ')
Kmag_col = RESULTS['FLUX_K']
print(Kmag_col)

print()
print('to get value: ')
Kmag_val = RESULTS['FLUX_K'][0]  #change index value to access other magnitudes if more than 1 row per object. 
print(Kmag_val)




to get table column: 
FLUX_K
 mag  
------
 13.54

to get value: 
13.54


# Loop through database

In [147]:
for index, row in db.iterrows():
    c = coordinates.SkyCoord(row['COORDS'], frame='icrs', unit='deg')
    RESULT = query_simbad(row['Source'], c)
    


object: 2M J11013205-7718249
RA, DEC: 11h01m31.992s -77d18m24.84s

SIMBAD results:
  MAIN_ID         RA           DEC      FLUX_K FLUX_ERROR_K FLUX_J FLUX_ERROR_J
               "h:m:s"       "d:m:s"     mag                 mag               
----------- ------------- ------------- ------ ------------ ------ ------------
ESO-HA  554 11 01 32.0412 -77 18 25.075  13.54        0.040 14.631        0.039
_______________________________________ o ________________________________________

object: 2MASS 1207-3932
RA, DEC: 12h07m00s -39d31m58.8s

No results found with SIMBAD
_______________________________________ o ________________________________________

object: 2MASS 1207-3932
RA, DEC: 12h07m00s -39d31m58.8s

No results found with SIMBAD
_______________________________________ o ________________________________________

object: 2MASS J04141188+2811535
RA, DEC: 04h14m11.8812s +28d11m53.5056s

SIMBAD results:
          MAIN_ID                 RA           DEC      ... FLUX_J FLUX_ERROR_J
   

/anaconda3/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '12:07:00 -39:31:58.8': No astronomical object found :
  (error.line, error.msg))
/anaconda3/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '12:07:00 -39:31:58.8': No astronomical object found :
  (error.line, error.msg))


SIMBAD results:
        MAIN_ID               RA           DEC      ... FLUX_J FLUX_ERROR_J
                           "h:m:s"       "d:m:s"    ...  mag               
----------------------- ------------- ------------- ... ------ ------------
2MASS J16085953-3856275 16 08 59.5393 -38 56 27.596 ... 13.902        0.029
           SONYC Lup3-6   16 08 59.52   -38 56 20.8 ...   17.4           --
_______________________________________ o ________________________________________

object: AKC2006-18
RA, DEC: 15h41m40.992s -33d45m19.08s

SIMBAD results:
        MAIN_ID               RA           DEC      ... FLUX_J FLUX_ERROR_J
                           "h:m:s"       "d:m:s"    ...  mag               
----------------------- ------------- ------------- ... ------ ------------
2MASS J15414081-3345188 15 41 40.8089 -33 45 18.862 ...  14.66        0.040
_______________________________________ o ________________________________________

object: CFHT 3
RA, DEC: 04h36m39s +22d58m12s

SIMBAD result

/anaconda3/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '11:06:16.992 -77:35:53.16': No astronomical object found :
  (error.line, error.msg))
/anaconda3/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '11:08:40.008 -77:34:51.96': No astronomical object found :
  (error.line, error.msg))
/anaconda3/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '11:08:40.008 -77:34:51.96': No astronomical object found :
  (error.line, error.msg))


No results found with SIMBAD
_______________________________________ o ________________________________________

object: Cha Ha 2
RA, DEC: 11h07m43.008s -77d33m59.04s

SIMBAD results:
 MAIN_ID        RA           DEC      FLUX_K FLUX_ERROR_K FLUX_J FLUX_ERROR_J
             "h:m:s"       "d:m:s"     mag                 mag               
--------- ------------- ------------- ------ ------------ ------ ------------
V* HK Cha 11 07 42.3440 -77 33 59.381 10.675        0.021  12.21        0.024
_______________________________________ o ________________________________________

object: Cha Ha 2
RA, DEC: 11h07m43.008s -77d33m59.04s

SIMBAD results:
 MAIN_ID        RA           DEC      FLUX_K FLUX_ERROR_K FLUX_J FLUX_ERROR_J
             "h:m:s"       "d:m:s"     mag                 mag               
--------- ------------- ------------- ------ ------------ ------ ------------
V* HK Cha 11 07 42.3440 -77 33 59.381 10.675        0.021  12.21        0.024
_____________________________________


SIMBAD results:
 MAIN_ID         RA           DEC      FLUX_K FLUX_ERROR_K FLUX_J FLUX_ERROR_J
              "h:m:s"       "d:m:s"     mag                 mag               
---------- ------------- ------------- ------ ------------ ------ ------------
[GY92]   3 16 26 21.9083 -24 44 39.720 10.857        0.023  12.34        0.023
_______________________________________ o ________________________________________

object: ISO-Oph 032
RA, DEC: 16h26m21.9084s -24d44m39.7208s

SIMBAD results:
 MAIN_ID         RA           DEC      FLUX_K FLUX_ERROR_K FLUX_J FLUX_ERROR_J
              "h:m:s"       "d:m:s"     mag                 mag               
---------- ------------- ------------- ------ ------------ ------ ------------
[GY92]   3 16 26 21.9083 -24 44 39.720 10.857        0.023  12.34        0.023
_______________________________________ o ________________________________________

object: ISO-Oph 032
RA, DEC: 16h26m21.9084s -24d44m39.7208s

SIMBAD results:
 MAIN_ID         RA          

SIMBAD results:
 MAIN_ID         RA           DEC      FLUX_K FLUX_ERROR_K FLUX_J FLUX_ERROR_J
              "h:m:s"       "d:m:s"     mag                 mag               
---------- ------------- ------------- ------ ------------ ------ ------------
[GY92] 312 16 27 38.9445 -24 40 20.672 12.286        0.025 16.541        0.127
_______________________________________ o ________________________________________

object: ISO-Oph 170
RA, DEC: 16h27m41.6102s -24d46m44.742s

SIMBAD results:
      MAIN_ID             RA           DEC      ... FLUX_J FLUX_ERROR_J
                       "h:m:s"       "d:m:s"    ...  mag               
------------------- ------------- ------------- ... ------ ------------
BKLT J162741-244645 16 27 41.6102 -24 46 44.742 ... 17.203        0.204
_______________________________________ o ________________________________________

object: ISO-Oph 175
RA, DEC: 16h27m45.7867s -24d44m53.862s

SIMBAD results:
      MAIN_ID             RA           DEC      ... FLUX_J F

No results found with SIMBAD
_______________________________________ o ________________________________________

object: SR12 c
RA, DEC: 16h27m19.992s -24d34m59.16s

No results found with SIMBAD
_______________________________________ o ________________________________________

object: SR12 c
RA, DEC: 16h27m19.992s -24d34m59.16s

No results found with SIMBAD
_______________________________________ o ________________________________________

object: SR12 c
RA, DEC: 16h27m19.992s -24d34m59.16s

No results found with SIMBAD
_______________________________________ o ________________________________________

object: SSPM J1102-3431
RA, DEC: 11h02m10.008s -34d30m36s

SIMBAD results:
MAIN_ID       RA           DEC      FLUX_K FLUX_ERROR_K FLUX_J FLUX_ERROR_J
           "h:m:s"       "d:m:s"     mag                 mag               
------- ------------- ------------- ------ ------------ ------ ------------
 TWA 28 11 02 09.8372 -34 30 35.573 11.887        0.024 13.034        0.024
__________

/anaconda3/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '16:27:19.992 -24:34:59.16': No astronomical object found :
  (error.line, error.msg))
/anaconda3/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '16:27:19.992 -24:34:59.16': No astronomical object found :
  (error.line, error.msg))
/anaconda3/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '16:27:19.992 -24:34:59.16': No astronomical object found :
  (error.line, error.msg))
/anaconda3/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribu